<p style='text-align:center'>
PSY 394U <b>Methods for fMRI</b>, Fall 2018


<img style='width: 300px; padding: 0px;' src='https://github.com/sathayas/JupyterfMRIFall2018/blob/master/Images/Placebo_Left.png?raw=true' alt='brain blobs'/>

</p>

<p style='text-align:center; font-size:40px; margin-bottom: 30px;'><b> Normalization </b></p>

<p style='text-align:center; font-size:18px; margin-bottom: 32px;'><b> October 1, 2018</b></p>

<hr style='height:5px;border:none' />

# Brief introduction to Nipype
<hr style="height:1px;border:none" />

* Steps in fMRI data analysis
* Why Nipype?

# Normalization - Theory
<hr style="height:1px;border:none" />

* What is normalization?
* Linear
* Non-linear
   * Various warping methods
* Cost function
* Regularization

# Normalization with Nipype
<hr style="height:1px;border:none" />

* With FSL
* With SPM
* With ANTs

# Normalization with a template
<hr style="height:1px;border:none" />

* Pediatric template
* Study-specific template

# Normalization of 
<hr style="height:1px;border:none" />

* Pediatric template
* Study-specific template